# 如何调试梯度

In [26]:
import numpy as np
import matplotlib.pyplot as plt

In [27]:
np.random.seed(666)
X = np.random.random(size=(1000,10))

In [28]:
true_theta = np.arange(1,12,dtype=float)
X_b = np.hstack((np.ones(shape=(len(X),1)),X))
y = X_b.dot(true_theta) + np.random.normal(size=1000)

In [29]:
X.shape

(1000, 10)

In [30]:
y.shape

(1000,)

In [31]:
true_theta

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.])

In [32]:
def J(theta, X_b, y):
    try:
        return np.sum((y - X_b.dot(theta))**2) / len(X_b)
    except:
        return float('inf')

In [33]:
# 数学推导的求梯度
def derivative_J_math(theta, X_b, y):
    return X_b.T.dot(X_b.dot(theta) - y) * 2. / len(y)

In [34]:
# 调试用的求梯度
def derivative_J_debug(theta, X_b, y, epsilon=0.01):
    res = np.empty(len(theta))
    for i in range(len(theta)):
        theta_1 = theta.copy()
        theta_1[i] += epsilon
        theta_2 = theta.copy()
        theta_2[i] -= epsilon
        res[i] = (J(theta_1, X_b, y) - J(theta_2, X_b, y)) / (2*epsilon)
    return res
        

In [35]:
# 批量梯度下降训练法
def gradient_descent(dJ, X_b, y, initial_theta, eta, n_iters=1e4, epsilon=1e-8):
    theta = initial_theta
    cur_iter = 0
    
    while cur_iter < n_iters:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta = theta - eta * gradient
        if (abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):
            break
        cur_iter += 1
    return theta

In [36]:
X_b = np.hstack((np.ones(shape=(len(X),1)),X))
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01

## 调试用梯度下降的效果

In [37]:
%time theta = gradient_descent(derivative_J_debug, X_b, y, initial_theta, eta)
theta

CPU times: user 6.97 s, sys: 2.86 s, total: 9.83 s
Wall time: 9.17 s


array([  1.1251597 ,   2.05312521,   2.91522497,   4.11895968,
         5.05002117,   5.90494046,   6.97383745,   8.00088367,
         8.86213468,   9.98608331,  10.90529198])

## 数学方式梯度下降的效果

In [38]:
%time theta = gradient_descent(derivative_J_math, X_b, y, initial_theta, eta)
theta

CPU times: user 833 ms, sys: 340 ms, total: 1.17 s
Wall time: 1.06 s


array([  1.1251597 ,   2.05312521,   2.91522497,   4.11895968,
         5.05002117,   5.90494046,   6.97383745,   8.00088367,
         8.86213468,   9.98608331,  10.90529198])

In [39]:
J(theta, X_b, y)

0.90497929349193762

In [40]:
J(true_theta,X_b,y)

0.91015768339662462

两者结果差不多，但调试用的方式慢许多